In [1]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, ret_count):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --endDeviceMaxRetransmissions="+f"{ret_count}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os

def run_scenario(traffic_type, realistic_model, device_class, scenario_name, ret_count):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, ret_count)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_confirmed_traffic_retransmission_attempts'
    test_sub_folder = str(ret_count)
    
    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []


In [7]:
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 2)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 4)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 8)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 16)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 32)))

In [8]:
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 2)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 4)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 8)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 16)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 32)))


In [9]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status confirmed_traffic_retransmission_attempts', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_CONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_CONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 2
ULPDR_CONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --re

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...


ULPDR_CONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=16 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=16 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 3
ULPDR_CONFIRMED_B, Simualtion with device count: 17
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=13 --dataUpType=1 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=2 --endDeviceType

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration


/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=32 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 3
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=32 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=3 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 4
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=8 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 3
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --endDeviceMaxRetransmissions=8 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=3 

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
0.9949748743718593 L: 0.24875
1.0 L: 0.30375
1.0 L: 0.3875
0.9917355371900827 L: 0.45375
0.9955456570155902 L: 0.56125
0.9893805309734514 L: 0.70625
0.9928774928774928 L: 0.8775
0.9882766705744431 L: 1.06625
0.9849624060150376 L: 1.33
0.9650721336370539 L: 1.64625
0.949605343047966 L: 2.05875
0.9015996122152206 L: 2.57875
0.8538884524744698 L: 3.1825
0.779773156899811 L: 3.9675
0.6862095335202651 L: 4.90375
0.5663973482494303 L: 6.03375
0.4507910074937552 L: 7.50625


Computing performance metrics...
Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.08125
1.0 L: 0.10375
0.9894736842105263 L: 0.11875
0.9915966386554622 L: 0.14875
1.0 L: 0.1825
0.9945054945054945 L: 0.2275
0.9868995633187773 L: 0.28625
0.9854014598540146 L: 0.3425
0.990909090909091 L: 0.4125
0.9850746268656716 L: 0.5025
0.9814432989690721 L: 0.60625
0.978688524590164 L: 0.7625
0.9878542510121457 L: 0.92625
0.965103598691385 L: 1.14625
0.9606087735004476 L: 1.39625
0.949964260185847 L: 1.74875
0.9264018691588785 L: 2.14
0.8822429906542056 L: 2.675
0.8139622641509434 L: 3.3125
0.7262195121951219 L: 4.1
0.61058648111332 L: 5.03
0.4949294094253331 L: 6.28625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
0.9933774834437086 L: 0.18875
0.9899497487437185 L: 0.24875
1.0 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9977777777777778 L: 0.5625
0.9858407079646018 L: 0.70625
0.9914529914529915 L: 0.8775
0.9882766705744431 L: 1.06625
0.9868421052631579 L: 1.33
0.9802581624905087 L: 1.64625
0.9562841530054644 L: 2.05875
0.9074163839069317 L: 2.57875
0.8723487824037707 L: 3.1825
0.7948960302457467 L: 3.9675
0.6770328829977058 L: 4.90375
0.5581106277190802 L: 6.03375
0.42964196502914237 L: 7.50625
46853.563806056976
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
